<a href="https://colab.research.google.com/github/yj9889/ESAA2/blob/main/220523%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%ED%8C%A8%ED%82%A4%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.8 파이썬추천시스템패키지
ch9. 8 pg. 607~624

## Surprise 패키지
: 파이썬 기반 추천 시스템 구축을 위한 전용 패키지

다양한 추천 알고리즘(사용자, 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재요인 협업 필터링 적용)

In [3]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 11.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633723 sha256=5016dd58186d8d3caebefa82413bbe01dc3dc14f084a89115f28fccf8b57b9ce
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:
import surprise 

print(surprise.__version__)

1.1.1


## **Surprise 를 이용한 추천 시스템 구축**
학습용, 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통한 잠재 요인 협업 필터링

In [5]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

surprise에서의 데이터 로딩은 *Dataset* 클래스를 이용해서만 가능

movies lens 데이터 세트의 userID, itemID, Rating 이 row level 형태

*load_builtin()*: 무비렌즈 사이트에서 제공하는 과거 버전 데이터 세트인 ml-100k or ml-1m 데이터를 내려받아 로컬 디렉터리에 저장한뒤 로딩

과거 데이터라 구분문자가 \t

surprise는 자체적으로 row level -> column level로 변경하므로 원본인 low level 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 함.

In [6]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


algo: 알고리즘 객체 생성

fit(학습 데이터 세트)을 수행해 학습 데이터 세트 기반으로 추천 알고리즘 학습

In [7]:
algo = SVD()
algo.fit(trainset) 

***test()***: 사용자-아이템 평점 데이터 세트 전체에 대해 추천을 예측하는 메서드(입력 데이터 세트에 대해 추천 데이터 세트 만듦)



* 리스트, 입력인자 데이터셋과 같은 25000개

* 25000개의 prediction 객체를 내부에 가짐

* prediction 객체는 surprise 패키지에서 제공하는 데이터 타입이며, 각 변수 정보에 기반해 surprise의 추천 예측 평점 데이터를 튜플 형태로 가짐

* prediction 객체의 details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터 남기는 데 사용
  * was_impossible이 True: 예측값을 생성할 수 없는 데이터라는 의미

In [8]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.627840414061691, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.796952367316425, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.340147565371415, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.755409336184906, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5432617309229912, details={'was_impossible': False})]

In [9]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.627840414061691),
 ('882', '291', 3.796952367316425),
 ('535', '507', 4.340147565371415)]

***predict()***: 개별 사용자와 영화에 대한 추천 평점(est) 반환 (test()는 predict()를 반복 입력한 것)

* 인자: 개별 사용자 아이디, 아이템 아이디
* 반환: 추천 예측 평점을 포함한 정보
	* 기존 평점 정보(r_ui)는 선택 사항, 아이디들은 문자열로 입력해야

In [10]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.29   {'was_impossible': False}


In [11]:
accuracy.rmse(predictions)

RMSE: 0.9488


0.9487514585849152

## **Surprise 주요 모듈 소개**

데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 3줄만 되어있어야 함.

* *Dataset.load_builtin(name='ml-100k')*
* *Dataset.load_from_file(file_path, reader)* : OS파일에서 데이터 로딩/콤마, 탭/(OS파일명, 파일의 포맷)
* *Dataset.load_from_df(df, reader)* : 판다스 df에서 로딩/(df객체, 파일의 포맷)

In [12]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/data/0523/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/drive/MyDrive/data/0523/ratings_noh.csv', index=False, header=False)

ratings_noh.csv: ratings.csv에서 헤더가 삭제된 파일

reader 객체 생성 시 line_format 인자로 user, item, rating, timestamp 4개의 칼럼으로 데이터 구성 명시 -> 후에 timestamp 제외

각 칼럼의 분리 문자는 콤마/평점 단위 .5/최대 평점 5

* Reader 클래스의 주요 생성 파라미터
	* line_format(string): 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
	* sep(char): 칼럼을 분리하는 분리자, 디폴드:\t, 판다스 df에서 입력받을 경우 기재 X
	* rating_scale(tuple, optional): 평점 값의 최소~최대 평점 설정

In [13]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/data/0523/ratings_noh.csv',reader=reader)

In [16]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [17]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/data/0523/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

* SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
* KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
* BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

* Surprise SVD의 비용함수: 사용자 베이스라인 평향성을 감안한 평점 예측에 regularization 적용
* SVD 클래스의 입력 파라미터
	* n_factors: 잠재요인 K의 개수, 정확도 높아지지만 과적합 문제
	* n_epochs: SGD 수행 반복 횟수
	* biased(bool): 베이스라인 사용자 편향 적용 여부

성능 평가 수치: SVD++ > SVD > K-NN Baseline

* 베이스라인 평점
  * 전체 평균 평점+사용자 편향 점수(사용자별 아이템 평점 평균 값-전체평균평점)+아이템 편향 점수(아이템별 평점 평균 값-전체평균평점)
  * : 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점 부과

## **교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝**

*cross_validate()*: 출력 결과와 폴드별 성능 평가 수치, 전체 폴드의 평균 성능 평가 수치 보여줌

In [19]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/drive/MyDrive/data/0523/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8731  0.8779  0.8766  0.8732  0.8687  0.8739  0.0032  
MAE (testset)     0.6705  0.6712  0.6709  0.6699  0.6720  0.6709  0.0007  
Fit time          5.01    6.12    5.81    5.09    5.07    5.42    0.45    
Test time         0.25    0.37    0.24    0.14    0.24    0.25    0.07    


{'fit_time': (5.0134265422821045,
  6.116093873977661,
  5.805214881896973,
  5.090279579162598,
  5.0673487186431885),
 'test_mae': array([0.67053875, 0.67115808, 0.67087611, 0.66987683, 0.67204213]),
 'test_rmse': array([0.87308128, 0.87791579, 0.87664796, 0.87317056, 0.8686717 ]),
 'test_time': (0.25460243225097656,
  0.3704218864440918,
  0.2434532642364502,
  0.13574981689453125,
  0.23991632461547852)}

하이퍼 파라미터 최적화: SVD는 n_epochs와 n_factors 튜닝

In [20]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8767461443488663
{'n_epochs': 20, 'n_factors': 50}


## **Surprise 를 이용한 개인화 영화 추천 시스템 구축**

데이터 세트를 그대로 fit()에 적용하면 오류

In [23]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

DatasetAutoFolds 객체 생성한 뒤 build_full_trainset()메서드 호출하면 전체 데이터를 학습 데이터 세트로 만들 수 있음



In [24]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/data/0523/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [25]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

userid 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤 예측 평점 계산


In [26]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/drive/MyDrive/data/0523/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [27]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


userid 9는 전체 9742개 영화 중 46개에만 평점 매김 -> 추천 대상 영화는 9696개

인자: 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 추천 상위 N개 개수

In [28]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [29]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
